In [ ]:
# Downgrade langchain to the last stable version (v0.3) to support RetrievalQA
%pip install "langchain<1.0.0" "langchain-community<1.0.0" "langchain-core<1.0.0" "langchain-openai<1.0.0" "langchain-text-splitters<1.0.0"

In [ ]:
# ------------------------------------------------------------------
# CELL 1: SETUP, IMPORTS, AND CONFIGURATION
# ------------------------------------------------------------------

import os
import time
from dotenv import load_dotenv

# Import necessary LangChain components
from langchain_pinecone import PineconeVectorStore
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# 1. Load API Keys from .env file
load_dotenv()

# --- CONFIGURATION ---
# Must match the Index Name used in Milestone 1
INDEX_NAME = "legal-assistant"
# Must match the Embedding Model used in Milestone 1
EMBEDDING_MODEL = "all-MiniLM-L6-v2" 

# Check for keys
if not os.environ.get("OPENAI_API_KEY"):
    print("❌ Error: OPENAI_API_KEY not found in .env file.")
else:
    print("✅ API Keys loaded successfully.")

In [ ]:
# ------------------------------------------------------------------
# CELL 2: CONNECT TO PINECONE DATABASE
# ------------------------------------------------------------------

print(f"⏳ Loading embedding model: {EMBEDDING_MODEL}...")
# We use the same model as M1 to ensure vectors match
embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)

print(f"⏳ Connecting to Pinecone Index: {INDEX_NAME}...")
vectorstore = PineconeVectorStore(
    index_name=INDEX_NAME, 
    embedding=embeddings
)

print("✅ Successfully connected to Pinecone VectorStore.")

In [ ]:
# ------------------------------------------------------------------
# CELL 3: DATABASE VERIFICATION (Human Check)
# ------------------------------------------------------------------
test_query = "fundamental rights"
print(f"🔎 Running Test Query: '{test_query}'")

try:
    # We fetch just 1 document to prove the 'ingestion' worked
    raw_results = vectorstore.similarity_search(test_query, k=1)
    
    if raw_results:
        print("✅ Database Check Passed!")
        doc = raw_results[0]
        print(f"   • Found Document: {doc.metadata.get('document_title', 'Untitled')}")
        print(f"   • Source File:    {doc.metadata.get('source_file')}")
        print(f"   • Content Snippet: {doc.page_content[:100]}...")
    else:
        print("❌ Database Check Failed. No results found.")
        print("   STOP: Please check if Milestone 1 ingestion was successful.")

except Exception as e:
    print(f"❌ Connection Error: {e}")

In [ ]:
# ------------------------------------------------------------------
# CELL 4: BUILD LEGABOT RAG CHAIN
# ------------------------------------------------------------------

# 1. Setup Retriever (The Tool for the AI)
# We convert the vectorstore into a retriever interface
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5} # Retrieve top 5 most relevant chunks
)

# 2. Initialize OpenAI (The Brain)
print("⏳ Initializing ChatOpenAI (gpt-3.5-turbo)...")
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo", 
    temperature=0.1 # Low temperature = Precise, Factual Legal Answers
)

# 3. Define the System Prompt (Based on Project Doc Section 5)
legal_system_prompt = """
You are LegaBot, a precise legal research assistant. 
For any user query, use the following context to provide an answer.

GUIDELINES:
1. Retrieve and list the most relevant statute sections/judgment excerpts.
2. Provide a concise legal summary (3–5 sentences).
3. If the answer requires legal interpretation or could affect rights, clearly say you are not a lawyer and recommend consulting a qualified attorney.
4. Always prioritize citing the exact statutory language or judgment excerpt used.
5. If you don't know the answer based on the context, say "I don't have enough information in my legal database."

CONTEXT:
{context}

QUESTION:
{question}

YOUR ANSWER:
"""

PROMPT = PromptTemplate(
    template=legal_system_prompt, 
    input_variables=["context", "question"]
)

# 4. Construct the Chain
# Connects: Retriever -> Prompt -> LLM
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True, # Required to show sources
    chain_type_kwargs={"prompt": PROMPT}
)

print("✅ LegaBot Chain built successfully. Ready to answer.")

In [ ]:
# ------------------------------------------------------------------
# CELL 5: RUN LEGABOT (With Detailed Top 5 Text Answers)
# ------------------------------------------------------------------

# Define your question
user_query = "What is the punishment for theft under Indian law?"

print(f"❓ USER QUESTION: '{user_query}'")
print("🤖 LEGABOT IS THINKING...")

# Run the chain
start_time = time.time()
response = qa_chain.invoke({"query": user_query})
end_time = time.time()

# --- 1. DISPLAY THE AI'S MAIN SUMMARY ---
print("\n" + "="*60)
print(f"📝 LEGABOT FINAL ANSWER ({end_time - start_time:.2f}s):")
print("="*60)
print(response["result"].strip())

# --- 2. DISPLAY THE TOP 5 RETRIEVED TEXTS (THE "5 RESPONSES") ---
print("\n" + "="*60)
print("🔍 TOP 5 RETRIEVED SOURCES (The Exact Text Found):")
print("="*60)

# Loop through the source docs and print the ACTUAL TEXT content
for i, doc in enumerate(response["source_documents"]):
    title = doc.metadata.get('document_title', 'Unknown Title')
    source = doc.metadata.get('source_file', 'Unknown File')
    
    print(f"\n[{i+1}] SOURCE: {title}")
    print(f"    File: {source}")
    # ⬇️ THIS IS THE NEW PART: Printing the actual text content ⬇️
    clean_text = " ".join(doc.page_content.split())
    print(f"    📖 EXCERPT: \"{clean_text}...\"")
    print("-" * 60)